In [1]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from ggplot import *
%matplotlib inline

In [2]:
file_list = glob.glob('nypd-sqf-data/*.csv')
# columns = ['year', 'datestop', 'timestop', 'frisked', 'searched', 'contrabn', 'sex', 'race', 'age', 'xcoord', 'ycoord']
columns = ['year', 'datestop', 'timestop', 'sex', 'race', 'age', 'city', 'xcoord', 'ycoord']

df = pd.DataFrame()
for file in file_list:
#     print("Loading into dataframe: ", file)
    frame = pd.read_csv(file, usecols=columns, na_values=' ', low_memory=False, nrows = 5000)
    frame['yearly_total'] = len(frame)
    df = pd.concat([df, frame], ignore_index=True)

df.head(5)

,age,city,datestop,race,sex,timestop,xcoord,ycoord,year,yearly_total
0,16,NaN,1012003,B,M,03:00,NaN,NaN,2003,5000
1,15,NaN,1012003,B,M,03:00,NaN,NaN,2003,5000
2,39,NaN,1012003,Q,M,03:00,NaN,NaN,2003,5000
3,20,NaN,1012003,Q,F,16:00,NaN,NaN,2003,5000
4,19,NaN,1022003,B,M,03:35,NaN,NaN,2003,5000


In [3]:
# Clean Age Column - Remove ages >100, and <3.
df['age'] = pd.to_numeric(df.age, errors='coerce')\
              .apply(lambda x: np.nan if x > 100 else np.nan if x < 3 else x )    

In [5]:
# Drop missing datestop values, parse 'datestop' into datetime format
df = df.dropna(subset=['datestop'])

add_zero = lambda x: str(x).zfill(8)
df['datetime'] = pd.to_datetime(df['datestop'].map(add_zero), format="%m%d%Y", errors='coerce')
df.loc[df['datetime'].isnull(),'datetime'] = pd.to_datetime(df.loc[df['datetime'].isnull(),'datestop'])

# Add column of only year & month
df['month_year'] = df['datetime'].map(lambda x: 100*x.year + x.month)


# Add monthly_count column (total of stops in a month)
for month in df.month_year.unique():
    df.loc[(df['month_year'] == month), 'monthly_total'] = len(df.loc[(df['month_year'] == month)])

In [6]:
df.sample(5)

,age,city,datestop,race,sex,timestop,xcoord,ycoord,year,yearly_total,datetime,month_year,monthly_total
31131,29,BROOKLYN,1022009,B,M,45,NaN,NaN,2009,5000,2009-01-02,200901,4933
27027,17,STATEN ISLAND,1022008,W,M,2154,942111,143650,2008,5000,2008-01-02,200801,4912
21735,33,MANHATTAN,1022007,Q,M,1938,987078,215157,2007,5000,2007-01-02,200701,4953
6083,17,NaN,3312004,B,M,00:00,NaN,NaN,2004,5000,2004-03-31,200403,536
45727,41,STATEN IS,1012012,Q,M,220,961596,166608,2012,5000,2012-01-01,201201,4758


In [7]:
# Races of ~90% of people stopped
cols = ['race', 'month_year', 'monthly_total']
races = df.loc[(df.race == 'B') | (df.race == 'P') | (df.race == 'Q') | (df.race == 'W'), cols].copy()
races['race'] = races['race'].map({'B':'B','W':'W','P':'H','Q':'H'})
races.head()

,race,month_year,monthly_total
0,B,200301,4880
1,B,200301,4880
2,H,200301,4880
3,H,200301,4880
4,B,200301,4880


In [12]:
races.groupby(['month_year', 'race']).count()

monthly_total
month_year race               
200301     B              2236
           H              1375
           W               580
200302     B                 9
           H                 6
           W                 1
200303     B                 4
           H                 5
           W                 3
200304     B                11
           H                 5
           W                 1
200305     B                 4
           H                 1
200306     B                 1
           H                 3
200307     B                 5
           H                 3
           W                 1
200308     B                14
           H                 1
           W                 2
200309     B                13
           H                 7
           W                 1
200311     B                 1
           H                 1
200312     W                 1
200401     B               239
           H               112
...                        ...
201001     B              2872
           H              1398
           W               425
201002     B                 5
           W                 1
201003     B                 6
           H                 2
201101     B              2538
           H              1637
           W               433
201201     B              2529
           H              1489
           W               444
201202     B                92
           H                20
201203     B               110
           H                19
           W                 1
201301     B              2782
           H              1407
           W               535
201401     B              2710
           H              1339
           W               546
201402     B                34
           H                14
           W                 6
201403     B                 4
           H                 4
           W                 1

[236 rows x 1 columns]